<div><a href="https://knodis-research-group.github.io/"><img style="float: right; width: 128px; vertical-align:middle" src="https://knodis-research-group.github.io/knodis-logo_horizontal.png" alt="KNODIS logo" /></a>

# Clasificación de texto con RNN<a id="top"></a>

<i><small>Última actualización: 2025-03-14</small></i></div>

***

## Introducción

En un _notebook_ anterior exploramos la clasificación de textos con CNN, las cuales son adecuadas para capturar dependencias locales en datos de texto. Sin embargo, a veces necesitamos un modelo más potente que pueda capturar dependencias a largo plazo en los datos. Aquí es donde entran en juego las RNN.

Las RNN están diseñadas específicamente para modelar datos secuenciales, lo que las hace ideales para tareas de clasificación de texto. A diferencia de las redes neuronales tradicionales, que procesan las entradas independientemente unas de otras, las RNN mantienen una memoria de las entradas anteriores y utilizan esta información para hacer predicciones sobre la entrada actual.

## Objetivos

Vamos a explorar cómo utilizar RNN para la clasificación de texto en el mismo problema que en el _notebook_ donde clasificábamos las reseñas de Amazon que los usuarios hicieron sobre los productos mediante CNN, pero esta vez con RNN.

Veremos que en realidad los cambios son mínimos, ya que es poco más que cambiar una capa por otra.

## Bibliotecas y configuración

A continuación importaremos las bibliotecas que se utilizarán a lo largo del cuaderno.

In [ ]:
import collections
import gzip
import re
import requests
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import torch
import torchmetrics

import utils

También configuraremos algunos parámetros para adaptar la presentación gráfica.

In [ ]:
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

Y crearemos los directorios necesarios en caso de que no se hayan creado previamente

In [ ]:
TEMP_PATH = pathlib.Path('tmp')
TEMP_PATH.mkdir(exist_ok=True)

***

## Parámetros comunes

Mantendremos los mismos parámetros globales para poder comparar ambos métodos.

In [ ]:
N_EPOCHS = 20           # Número de iteraciones de entrenamiento
BATCH_SIZE = 32768      # Número de ejemplos por batch
EMBEDDING_DIM = 50      # Dimensiones de nuestro embedding (50, 100, 200 o 300)
MAX_VOCAB_SIZE = 20000  # Tamaño máximo de nuestro vocabulario
MAX_SEQUENCE_LEN = 128  # Longitud máxima de las secuencias de palabras

## Preprocesamiento de datos

El proceso que llevaremos a cabo será el mismo que hicimos en el _notebook_ anterior.

### Descarga del _dataset_ ...

In [ ]:
DATASET_URL = 'https://github.com/blazaid/aprendizaje-profundo/raw/refs/heads/gh-pages/Datasets/Digital_Music_5.json.gz'
DATASET = pathlib.Path('tmp/Digital_Music_5.json')

print(f"Downloading dataset to {DATASET.resolve()}")
if not DATASET.exists():
    with requests.get(DATASET_URL, stream=True) as response:
        response.raise_for_status()
        with gzip.GzipFile(fileobj=response.raw) as f_gz:
            with DATASET.open("wb") as f:
                shutil.copyfileobj(f_gz, f)
else:
    print("File already exists! Nice")

print("Loading text corpus")
corpus = pd.read_json(DATASET, lines=True)
corpus.dropna(subset=['overall', 'reviewText'], inplace=True)
corpus.head()

print("Done")

### ... preparación de las entradas y las salidas ...

In [ ]:
x_train = corpus['reviewText'].astype(str).str.strip()
y_train = corpus['overall'].astype(int).replace({
    1: 0,
    2: 0,
    3: 1,
    4: 2,
    5: 2,
})
num_classes = len(set(y_train))

print(f"Training input shape:  {x_train.shape}")
print(f"Training output shape: {y_train.shape}")
print(f"No. of classes:        {num_classes}")

### ... _tokenización_, _datasets_ y _dataloaders_ ...

In [ ]:
def tokenizer(text):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b|[^\w\s]', text, re.UNICODE)
    return tokens


counter = collections.Counter()
for text in x_train:
    counter.update(tokenizer(text))


special_tokens = ['<PAD>', '<UNK>']

most_common = counter.most_common(MAX_VOCAB_SIZE - len(special_tokens))
vocab_words = [word for word, _ in most_common]

vocab = {token: i for i, token in enumerate(special_tokens)}
for i, word in enumerate(vocab_words, start=len(special_tokens)):
    vocab[word] = i

vocab_size = len(vocab)
print(f'Tamaño del vocabulario: {vocab_size}')


def text_to_sequence(text, vocab, max_len=MAX_SEQUENCE_LEN):
    tokens = tokenizer(text)
    seq = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    if len(seq) > max_len:
        seq = seq[:max_len]
    else:
        seq = seq + [vocab['<PAD>']] * (max_len - len(seq))
    return seq


class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, vocab, max_len=MAX_SEQUENCE_LEN):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.vocab = vocab
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        seq = text_to_sequence(self.texts[idx], self.vocab, self.max_len)
        return torch.tensor(seq, dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)


dataset = TextDataset(x_train, y_train, vocab, MAX_SEQUENCE_LEN)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

### ... los _embeddings_ preentrenados de GloVe ...

In [ ]:
GLOVE_URL = 'http://nlp.stanford.edu/data/glove.6B.zip'
GLOVE_FILE = pathlib.Path('tmp/glove.6B.zip')

# Download the compressed GloVe dataset (if you don't already have it)
if not GLOVE_FILE.exists():
    print('Downloading GloVe ...', end='')
    with open(GLOVE_FILE, 'wb') as f:
        r = requests.get(GLOVE_URL, allow_redirects=True)
        f.write(r.content)
    print('OK')

# Unzip it in the directory 'glove'.
print('Unpacking ...', end='')
shutil.unpack_archive(GLOVE_FILE, 'tmp')
print('OK')

### ... cogiendo los pesos de las `MAX_VOCAB_SIZE` palabras más comunes ...

In [ ]:
print(f'Loading GloVe {EMBEDDING_DIM}-d embedding... ', end='')
word2vec = {}
with open(f'tmp/glove.6B.{EMBEDDING_DIM}d.txt') as f:
    for line in f:
        word, vector = line.split(maxsplit=1)
        word2vec[word] = np.fromstring(vector,'f', sep=' ')
print(f'done ({len(word2vec)} word vectors loaded)')

print('Creating embedding matrix with GloVe vectors... ', end='')

# Our newly created embedding: a matrix of zeros
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

ko_words = 0
for word, idx in vocab.items():
    word_glove = 'unk' if word == '<UNK>' else word
    vector = word2vec.get(word_glove)
    if vector is not None:
        embedding_matrix[idx] = vector
    else:
        ko_words += 1

print(f'done ({ko_words} out of {len(vocab)} words unassigned)')

## Clasificación basada en redes neuronales recurrentes

Y ahora, en lugar de utilizar CNN, utilizaremos RNN. En este caso, el conjunto de dimensiones lo realizan la capa `TextVectorization` (que convierte el texto en secuencias de enteros de longitud $T$) y la capa Embedding (que convierte cada entero en un vector de dimensiones $D$), convirtiendo la entrada en un tensor con la forma $N \times T \times D$.

In [ ]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes, embedding_matrix, bidirectional=True, dropout=0.5):
        super().__init__()

        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = torch.nn.Parameter(
            torch.tensor(embedding_matrix),
            requires_grad=False,
        )
        
        self.lstm = torch.nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional,  # Si usamos bidireccional o no
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.dropout = torch.nn.Dropout(dropout)

        self.fc = torch.nn.Linear(
            hidden_dim * (2 if bidirectional else 1),   # ¡Ojo! Bidireccional
                                                        #  dobla las entradas
            num_classes,
        )
    
    def forward(self, x):
        x = self.embedding(x)  # (B, seq_len, embedding_dim)
        x = x.float()
        out, _ = self.lstm(x)  # (B, seq_len, hidden_dim * embedding_dim)
        out = out.transpose(1, 2)  # (B, hidden_dim * num_directions, seq_len)
        # Max pooling adaptativo a lo largo del tiempo para un vector/muestra
        pooled = torch.nn.functional.adaptive_max_pool1d(out, 1).squeeze(2)
        pooled = self.dropout(pooled)
        logits = self.fc(pooled)
        return logits


rnn_model = RNNClassifier(
    vocab_size,
    EMBEDDING_DIM,
    hidden_dim=32,
    num_layers=2,
    num_classes=num_classes,
    embedding_matrix=embedding_matrix,
    bidirectional=True,
    dropout=0.5,
)
print(rnn_model)

Aunque el número de parámetros es similar, aumentar el número de unidades en una unidad recurrente no aumenta mucho el número de parámetros en nuestro modelo. Sin embargo, sí que aumentará mucho el tiempo de entrenamiento. Por lo tanto, nuestro modelo no podrá obtener resultados tan buenos como los anteriores.

Entrenemos el modelo y esperemos que todo vaya bien (otra vez)

In [ ]:
history = utils.train(
    rnn_model,
    train_loader,
    n_epochs=N_EPOCHS,
    criterion=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(rnn_model.parameters()),
    metric_fn=torchmetrics.classification.MulticlassAccuracy(num_classes=num_classes),
)

Echemos un vistazo al progreso del entrenamiento:

In [ ]:
pd.DataFrame(history).plot()
plt.yscale('log')
plt.xlabel('Epoch num.')
plt.show()

Veamos ahora cómo interpreta el sentimiento de una reseña buena, regular y mala extraída del sitio web de amazon.

In [ ]:
good = ("My nephew is on the autism spectrum and likes to fidget with things so I knew this toy would be a hit. "
        "Was concerned that it may not be \"complex\" enough for his very advanced brain but he really took to it. "
        "Both him (14 yrs) and his little brother (8 yrs) both enjoyed playing with it throughout Christmas morning. "
        "I'm always happy when I can find them something unique and engaging.")

poor = ("I wasn't sure about this as it's really small. I bought it for my 9 year old grandson. "
        "I was ready to send it back but my daughter decided it was a good gift so I'm hoping he likes it. "
        "Seems expensive for the price though to me.")

evil = ("I just wanted to follow up to say that I reported this directly to the company and had no response. "
        "I have not gotten any response from my review. The level of customer service goes a long way when an item "
        "you purchase is defective and this company didn’t care to respond. No I am even more Leary about ordering "
        "anything from this company. I never asked for a refund or replacement since I am not able to return it. "
        "I’m just wanted to let them know that this was a high dollar item and I expected it to be a quality item. "
        "Very disappointed! I bought this for my grandson for Christmas. He loved it and played with it a lot. "
        "My daughter called to say that the stickers were peeling on the corners. I am not able to take it from my "
        "grandson because he is autistic and wouldn’t understand. I just wanted to warn others who are wanting to get "
        "this. Please know that this is a cool toy and it may not happen to yours so it is up to you.")

def predict_text(text, model, vocab):
    model.eval()
    seq = text_to_sequence(text, vocab, MAX_SEQUENCE_LEN)
    seq_tensor = torch.tensor(seq, dtype=torch.long).unsqueeze(0)  # crear batch de 1
    with torch.no_grad():
        logits = model(seq_tensor)
        pred = logits.argmax(dim=1).item()
    return pred

print(f'Good was classified as {predict_text(good, cnn_model, vocab)}')
print(f'Poor was classified as {predict_text(poor, cnn_model, vocab)}')
print(f'Evil was classified as {predict_text(evil, cnn_model, vocab)}')

## Conclusiones

Hemos estudiado cómo utilizar RNN para clasificar textos y las hemos comparado con las CNN. Aunque ambas arquitecturas pueden ser eficaces para la clasificación de textos, presentan algunas diferencias clave.

La principal es las RNN son más adecuadas para captar las dependencias a largo plazo en los datos de texto, mientras que las CNN son más adecuadas para captar las dependencias locales. Esto hace que las RNN sean una buena opción para tareas como el análisis de sentimientos o la traducción de idiomas, donde el contexto de una palabra o frase puede ser crucial para determinar su significado.

Sin embargo, el entrenamiento de las RNN puede ser más costoso que el de las CNN (ya hemos visto el tiempo que tarda una red pequeña en ser entrenada durante 5 _epochs_). Por otro lado, las CNN son más rápidas de entrenar y se adaptan mejor a conjuntos de datos más grandes. Son especialmente adecuadas para tareas como la clasificación de textos o el reconocimiento de imágenes, en las que las características locales son importantes.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>